In [1]:
from contractions import contractions_dict

In [2]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import textwrap
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download("stopwords")
nltk.download("punkt")

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kysgattu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kysgattu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def expand_contractions(text, contraction_map=None):
    if contraction_map is None:
        contraction_map = contractions_dict

    # Using regex for getting all contracted words
    contractions_keys = '|'.join(re.escape(key) for key in contraction_map.keys())
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL)

    expanded_text = contractions_pattern.sub(lambda match: contraction_map.get(match.group(0), match.group(0)), text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


In [4]:
entertainment = pd.read_csv('data/BBCNewsSummaryCSV/entertainment_data.csv') 
sport = pd.read_csv('data/BBCNewsSummaryCSV/sport_data.csv')
tech = pd.read_csv('data/BBCNewsSummaryCSV/tech_data.csv')

In [5]:
print(entertainment.head())

            filename                                        newsarticle  \
0  entertainment_001  Gallery unveils interactive tree\n\nA Christma...   
1  entertainment_002  Jarre joins fairytale celebration\n\nFrench mu...   
2  entertainment_003  Musical treatment for Capra film\n\nThe classi...   
3  entertainment_004  Richard and Judy choose top books\n\nThe 10 au...   
4  entertainment_005  Poppins musical gets flying start\n\nThe stage...   

                                             summary  
0  The messages will be "unwrapped" by sculptor R...  
1  Bloom is to be formally presented with the Han...  
2  The classic film It's A Wonderful Life is to b...  
3  "It was very hard to follow last year's extrem...  
4  Mary Poppins was originally created by author ...  


In [6]:
training_dataset = pd.concat([entertainment,sport,tech], ignore_index=True)

In [7]:
training_dataset = training_dataset.sample(frac=1).reset_index(drop=True)

In [8]:
training_dataset = training_dataset.replace([np.inf, -np.inf], np.nan).dropna()

In [9]:
training_dataset.head()

,filename,newsarticle,summary
0,tech_250,Microsoft sets sights on spyware\n\nWindows us...,Windows users could soon be paying Microsoft t...
1,entertainment_262,"U2's desire to be number one\n\nU2, who have w...",But they still want more.They have to want to ...
2,sport_008,Hansen 'delays return until 2006'\n\nBritish t...,Moore said Hansen may be able to return to spr...
3,sport_056,Kluft impressed by Sotherton form\n\nOlympic h...,"Sotherton has also displayed promise, with a n..."
4,tech_372,Text message record smashed again\n\nUK mobile...,"Before December's bumper text record, the prev..."


In [10]:
stop_words = stopwords.words("english")

In [11]:
def wrap(x):
    return textwrap.fill(x,replace_whitespace=False,fix_sentence_endings=True)

In [12]:
def summarize_textrank(text,factor=0.15):
    # extract sentences
    sents = sent_tokenize(text)
    # perform tfidf
    tfidf = TfidfVectorizer(stop_words=stop_words,norm='l1')
    X = tfidf.fit_transform(sents)
    
    # compute similarity matrix
    S = cosine_similarity(X)
    # Normalize
    S  = S/S.sum(axis=1,keepdims=True)
    
    # Uniform transition matrix
    U = np.ones_like(S)/len(S)
    
    # smmothed similarity matrix
    S = (1-factor) * S + factor * U
    
    # find the limiting/stationary distribution
    eigenvals,eigenvecs = np.linalg.eig(S.T)
    
    # compute scores
    scores = eigenvecs[:,0]/eigenvecs[:,0].sum()
    
    # sort the scores
    sort_idx = np.argsort(-scores)
    
    # print summary
    
    print("Summary:\n")
    for i in sort_idx[:5]:
        print(wrap("%.2f: %s"%(scores[i],sents[i])))

In [27]:
summarize_textrank(training_dataset['newsarticle'][1])

Summary:

0.03: But they still want more.
0.03: The band have done "everything in their considerable powers" to
ensure they remain the biggest band in the world, according to Q
magazine editor Paul Rees.
0.03: We don't want to be thought of as a veteran band."
0.03: They have to want to be the biggest band ever and not stop
wanting it.
0.03: They also have to want it.


In [26]:
training_dataset['TextRankSummary'] = training_dataset['newsarticle'][1].apply(summarize_textrank)

AttributeError: 'str' object has no attribute 'apply'

In [14]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [36]:
def textRank(text):
    sentences=sent_tokenize(text)
    sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
    stop_words = stopwords.words('english')
    sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
    w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)
    sentence_embeddings=[[w2v.wv[word][0] for word in words] for words in sentence_tokens]
    max_len=max([len(tokens) for tokens in sentence_tokens])
    sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
    similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
    for i,row_embedding in enumerate(sentence_embeddings):
        for j,column_embedding in enumerate(sentence_embeddings):
            similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
    top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
    summary = ''
    for sent in sentences:
        if sent in top.keys():
            summary = summary+sent
            # print(summary)
    return summary

In [37]:
textRank(training_dataset['newsarticle'][1])

'They have to be compelling performers, charismatic and intelligent enough to make good decisions and keep their feet on the ground."The challenge is to be bigger and bolder and better - to make records the whole world will listen to," Bono recently said.Drummer Larry Mullen Jr echoed those sentiments: "We\'re very competitive - we want to be on the radio, have big singles.The end of the decade marked a crucial point for the band - they had reached the top but still yearned for new challenges and achievements.'

In [38]:
print(training_dataset['summary'][1],"____\n_____",textRank(training_dataset['newsarticle'][1]))

But they still want more.They have to want to be the biggest band ever and not stop wanting it.The band have done "everything in their considerable powers" to ensure they remain the biggest band in the world, according to Q magazine editor Paul Rees.We don't want to be thought of as a veteran band.""The challenge is to be bigger and bolder and better - to make records the whole world will listen to," Bono recently said.U2, who have won three prestigious Grammy Awards for their hit Vertigo, are stubbornly clinging to their status as one of the biggest bands in the world.The end of the decade marked a crucial point for the band - they had reached the top but still yearned for new challenges and achievements.They are, perhaps, alone as the only rock band that has got better with age."They also have to want it.In 1987, The Joshua Tree broke sales records and saw the band reach the height of their powers with hits including Where the Streets Have No Name, I Still Haven't Found What I'm Look

In [39]:
dd = """Ajax refuse to rule out Jol move..Ajax have refused to reveal whether Tottenham's boss Martin Jol is on the Dutch champions' shortlist to become the Amsterdam club's new coach...Jol, who has coached in his native Holland, has guided Spurs to the Premiership's top eight. An Ajax spokesman told BBC Sport: "The coach must fit our profile - a coach who understands the Dutch league and offensive and distinctive football. "We need to find a solution soon, so someone is in place for next season." Ronald Koeman quit as Ajax boss last week after their exit from the Uefa Cup. Jol has been linked with the vacant post at Ajax, with reports saying he has fallen out with Spurs' sporting director Frank Arnesen. But in a statement on Spurs' website, Jol said: "I'm happy here, I'm not in discussion with anyone else, I don't want to go elsewhere." Ajax have enlisted the help of Dutch legend Johann Cruyff, currently a consultant at Barcelona, to help find a new head coach. Cruyff has admitted he has been impressed by the way former RFC Waalwijk coach Jol has turned round Spurs' fortunes since taking over from Jacques Santini. Tonny Bruins Slot and Ruud Krol are currently in charge of Ajax, who are third in the Dutch league."""

In [40]:
len(dd)

1224